Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

In [ ]:
!pip install minsearch

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [ ]:
question = 'Can I still join the course?'

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
search_results = search(question)

In [ ]:
prompt = build_prompt(question, search_results)

In [3]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



In [ ]:
answer = llm(prompt)

In [ ]:
print(answer)

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag("How do I patch KDE under FreeBSD?")

## "Agentic" RAG

In [ ]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [ ]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [ ]:
prompt = prompt_template.format(question=question, context=context)

In [ ]:
answer_json = llm(prompt)

In [4]:
import json

In [ ]:
answer = json.loads(answer_json)

In [ ]:
answer

In [ ]:
answer['action']

In [ ]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [ ]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [ ]:
answer_json = llm(prompt)

In [ ]:
print(answer_json)

In [ ]:
def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)

    return answer

In [ ]:
agentic_rag_v1('how do I join this course ?')

In [ ]:
agentic_rag_v1('how patch KDE under FreeBSD?')

## Agentic Search

In [ ]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [ ]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [ ]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [ ]:
context = build_context(search_results)

In [ ]:
context

In [ ]:
prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [ ]:
answer_json = llm(prompt)

In [ ]:
answer = json.loads(answer_json)

In [ ]:
answer

In [ ]:
previous_actions.append(answer)

In [ ]:
previous_actions

In [ ]:
keywords = answer['keywords']

In [ ]:
keywords

In [ ]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [ ]:
search_results = dedup(search_results)

In [ ]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [ ]:
answer_json = llm(prompt)

In [ ]:
answer = json.loads(answer_json)

In [ ]:
print(answer)

In [ ]:
question = "what do I need to do to be successful at this llm course?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

In [ ]:
answer

In [ ]:
iteration

## Function calling ("tool use")

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [6]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [7]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [8]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_wjhz6s3RI7gXtVR8SDSu1x9I', name='search', type='function_call', id='fc_68855f8c301881a2bfc63964e32361f60f4a361771e540a9', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"study tips for module 1"}', call_id='call_S6C1qhX3WqoTClxOI2Y2z4K9', name='search', type='function_call', id='fc_68855f8c620081a2b11aebf16e3357000f4a361771e540a9', status='completed')]

In [9]:
calls = response.output

In [10]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [11]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_68855fb3170081a2aae7e6ff1032fc130f4a361771e540a9', content=[ResponseOutputText(annotations=[], text="To excel in Module 1, consider the following strategies:\n\n1. **Understand the Basics**: Ensure you have a solid grasp of foundational concepts related to Docker and Terraform, as these are crucial for this module.\n\n2. **Practice Coding**: Engage with hands-on coding exercises. For instance, experiment with SQLAlchemy commands and make note of common errors (like the `TypeError: 'module' object is not callable`).\n\n3. **Address Common Errors**:\n   - If you encounter `ModuleNotFoundError: No module named 'psycopg2'`, make sure to install the necessary packages using:\n     ```bash\n     pip install psycopg2-binary\n     ```\n\n4. **Explore Resources**: Utilize the course materials and documentation related to Docker and Terraform. This includes troubleshooting tips and best practices.\n\n5. **Collaborate with Peers**: Join study groups or discussion fo

In [16]:
for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.content[0].text) 

message
To excel in Module 1, consider the following strategies:

1. **Understand the Basics**: Ensure you have a solid grasp of foundational concepts related to Docker and Terraform, as these are crucial for this module.

2. **Practice Coding**: Engage with hands-on coding exercises. For instance, experiment with SQLAlchemy commands and make note of common errors (like the `TypeError: 'module' object is not callable`).

3. **Address Common Errors**:
   - If you encounter `ModuleNotFoundError: No module named 'psycopg2'`, make sure to install the necessary packages using:
     ```bash
     pip install psycopg2-binary
     ```

4. **Explore Resources**: Utilize the course materials and documentation related to Docker and Terraform. This includes troubleshooting tips and best practices.

5. **Collaborate with Peers**: Join study groups or discussion forums. Sharing challenges and solutions can enhance your understanding significantly.

6. **Utilize Online Forums**: If you're stuck, consi

In [17]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [18]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 What is this course about ? 


function_call: ResponseFunctionToolCall(arguments='{"query":"course overview"}', call_id='call_LfduO3NC8aFSgMzZUok6H5Ug', name='search', type='function_call', id='fc_68856062f6cc819d91c83ab984f425150ba22409d13868f8', status='completed')

The course is focused on Data Engineering, specifically designed to guide students through various tools and practices in this field. It includes learning about alternative data stacks, where you'll explore both GCP (Google Cloud Platform) and local installations. You'll have the chance to work with different tools and frameworks relevant to data engineering, and you'll also engage in peer reviews for projects.

Key details include:
- The course will start on January 15, 2024, at 17:00.
- After the course, you'll have access to all materials for self-paced learning.
- Certificates are only awarded to participants who finish the course in real-time with a live cohort.

If you have further questions about specific topics or tools covered, feel free to as

 what assignments are given as part of this course ? 


function_call: ResponseFunctionToolCall(arguments='{"query":"course assignments"}', call_id='call_Xu7BDhZ8TAplKIiE1gw15LMZ', name='search', type='function_call', id='fc_6885608112e8819d8226dbc22242dee20ba22409d13868f8', status='completed')

The FAQ didn’t provide specific details about the assignments for this course. However, I can share that typically, in a Data Engineering course, assignments may include:

1. **Hands-on Projects**: Implementing projects that involve data pipelines, using tools such as Airflow, GCP, or similar.
2. **Homework Assignments**: Tasks that reinforce learning from lectures, often involving coding exercises or data manipulation tasks.
3. **Capstone Project**: A final project that usually involves the application of the concepts and tools learned throughout the course, which will require peer review.

Since the course emphasizes practical skills, you'll want to be prepared for a combination of theoretical knowledge and hands-on experiences.

Would you like to

 Does this course teach DBT ?


...

 What Database they use ?


function_call: ResponseFunctionToolCall(arguments='{"query":"database used in course"}', call_id='call_Mesz7UHMmzOCe2BtGT9wECKq', name='search', type='function_call', id='fc_688560b705dc819dbd496d1b6beedc020ba22409d13868f8', status='completed')

The course uses multiple databases as part of its educational framework. Specifically, you will work with:

1. **PostgreSQL**: This is often utilized for various assignments and exercises, especially in a Docker environment.
2. **BigQuery**: Used in conjunction with dbt for analytics engineering tasks.

The flexibility of the course allows for the use of different databases and tools, providing a comprehensive understanding of data engineering.

If there's a specific database feature or tool you want to know more about, let me know! Which database are you most interested in learning more about?



 Bigquery ?


function_call: ResponseFunctionToolCall(arguments='{"query":"BigQuery in course"}', call_id='call_Jt70PZBkNe4pXkcIxeVUSJtE', name='search', type='function_call', id='fc_688560d10084819da673018cdaa0e48f0ba22409d13868f8', status='completed')

The course incorporates **Google BigQuery** as a significant part of the curriculum. Here’s what you can expect with BigQuery:

1. **Analytics Engineering**: You'll use BigQuery in conjunction with dbt to build and transform datasets for analysis.
2. **Hands-On Practice**: There will be practical assignments that allow you to interact with BigQuery, focusing on querying large datasets efficiently and effectively.
3. **Error Handling**: The course also addresses common issues encountered when working with BigQuery, such as data type mismatches and connection errors.

This integration provides you with valuable experience in working with one of the industry's leading cloud data warehouses.

If you have specific questions about working with BigQuery or

 stop


## Multiple tools

In [19]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-26 23:12:35--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.1’

chat_assistant.py.1 100%[===================>]   3.41K  --.-KB/s    in 0s      

2025-07-26 23:12:35 (49.5 MB/s) - ‘chat_assistant.py.1’ saved [3495/3495]



In [20]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [21]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [22]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

In [23]:
tools.add_tool(add_entry, add_entry_description)

In [24]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [25]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [26]:
chat.run()

You: What is this course About ? 


You: Do I need to use AWS or GCP ?


You: stop


Chat ended.


In [ ]:
index

In [ ]:
index.docs[-1]

In [27]:
pip install pydantic-ai

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 37.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/32 [uvicorn]
    Found existing installation: tenacity 9.1.27m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/32 [uvicorn]
    Uninstalling tenacity-9.1.2:m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/32 [uvicorn]
      Successfully uninstalled tenacity-9.1.2237m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/32 [uvicorn]
  Attempting uninstall: pydantic-cor

In [29]:
from pydantic_ai import Agent, RunContext

In [30]:
chat_agent = Agent(  
    'openai:gpt-4o-mini',
    system_prompt=developer_prompt
)

In [31]:
from typing import Dict


@chat_agent.tool
def search_tool(ctx: RunContext, query: str) -> Dict[str, str]:
    """
    Search the FAQ for relevant entries matching the query.

    Parameters
    ----------
    query : str
        The search query string provided by the user.

    Returns
    -------
    list
        A list of search results (up to 5), each containing relevance information 
        and associated output IDs.
    """
    print(f"search('{query}')")
    return search(query)


@chat_agent.tool
def add_entry_tool(ctx: RunContext, question: str, answer: str) -> None:
    """
    Add a new question-answer entry to FAQ.

    This function creates a document with the given question and answer, 
    tagging it as user-added content.

    Parameters
    ----------
    question : str
        The question text to be added to the index.

    answer : str
        The answer or explanation corresponding to the question.

    Returns
    -------
    None
    """
    return add_entry(question, answer)

In [32]:
user_prompt = "I just discovered the course. Can I join now?"
agent_run = await chat_agent.run(user_prompt)
print(agent_run.output)

search('course enrollment')
Yes, you can still join the course! However, it's recommended to register before the official start date, which is on January 15, 2024, at 17:00. If you want to join, make sure to follow the registration link provided by the course.

Would you like the link to register or need help with anything else?
